In [ ]:
%pip install -U llama-index-core

In [ ]:
%pip install llama-index-embeddings-huggingface llama-agents

In [ ]:
import logging

# Create a logger instance
logger = logging.getLogger(__name__)

# Set the log level to INFO for informational messages
logger.setLevel(logging.INFO)

In [ ]:
import nest_asyncio

nest_asyncio.apply()

#import os

#os.environ["OPENAI_API_KEY"] = "sk-proj-..."

In [15]:
from llama_agents import (
    AgentService,
    AgentOrchestrator,
    ControlPlaneServer,
    LocalLauncher,
    SimpleMessageQueue,
)

from llama_index.core.agent import ReActAgent
from llama_index.core.tools import FunctionTool
from llama_index.llms.ollama import Ollama
from llama_index.core import Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.agent import FunctionCallingAgentWorker


# Calculator tool 
def calculator(operation: str, a: float, b: float) -> str:
    """
    Perform basic arithmetic operations.
    
    Args:
    operation (str): One of 'add', 'subtract', 'multiply', or 'divide'.
    a (float): First number.
    b (float): Second number.
    
    Returns:
    str: Result of the operation as a string.
    """
    try:
        if operation == "add":
            result = a + b
        elif operation == "subtract":
            result = a - b
        elif operation == "multiply":
            result = a * b
        elif operation == "divide":
            if b == 0:
                return "Error: Cannot divide by zero."
            result = a / b
        else:
            return f"Error: Invalid operation '{operation}'. Choose 'add', 'subtract', 'multiply', or 'divide'."
        
        return f"The result of {a} {operation} {b} is {result}"
    except Exception as e:
        return f"Error in calculation: {str(e)}"

# New text analysis tool
def text_analyzer(text: str) -> str:
    """
    Perform basic text analysis.
    
    Args:
    text (str): The text to analyze.
    
    Returns:
    str: Analysis results as a string.
    """
    try:
        word_count = len(text.split())
        char_count = len(text)
        sentence_count = text.count('.') + text.count('!') + text.count('?')
        
        analysis = f"Text Analysis Results:\n"
        analysis += f"- Word count: {word_count}\n"
        analysis += f"- Character count: {char_count}\n"
        analysis += f"- Approximate sentence count: {sentence_count}\n"
        
        return analysis
    except Exception as e:
        return f"Error in text analysis: {str(e)}"

calculator_tool = FunctionTool.from_defaults(fn=calculator)
text_tool = FunctionTool.from_defaults(fn=text_analyzer)


llm = Ollama(model="arcee-ai/arcee-agent", request_timeout=120.0)

Settings.llm = llm



agent1 = ReActAgent.from_tools(tools=[calculator_tool], llm=llm, verbose=True)
agent2 = ReActAgent.from_tools([text_tool], llm=Ollama(model="mistral", request_timeout=120.0), verbose=True)

# worker2 = FunctionCallingAgentWorker.from_tools([text_tool], llm=llm)
# agent2 = worker2.as_agent()


Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5"
)

# create our multi-agent framework components
message_queue = SimpleMessageQueue(port=8000)
control_plane = ControlPlaneServer(
    message_queue=message_queue,
    orchestrator=AgentOrchestrator(llm=llm),
    port=8001,

)
agent_server_1 = AgentService(
    agent=agent1,
    message_queue=message_queue,
    description="Useful for performing basic arithmetic operations like calculations.",
    service_name="calculator_agent",
    port=8002,
)
agent_server_2 = AgentService(
    agent=agent2,
    message_queue=message_queue,
    description="Useful for performing NLP, Text Analysis and Text Processing.",
    service_name="nlp_agent",
    port=8003,
)

# launch it
launcher = LocalLauncher([agent_server_1, agent_server_2], control_plane, message_queue)

try:
    result = launcher.launch_single("can you divide 100 by 20?")


    print(f"Result: {result}")
except Exception as e:
    print(f"An error occurred: {str(e)}")
    print("Please check the agent logs for more details.")

INFO:llama_agents.message_queues.simple - Consumer AgentService-de9efec0-c3f8-44bd-90ce-6762078c7852: calculator_agent has been registered.
INFO:llama_agents.message_queues.simple - Consumer AgentService-18c874f9-d0ad-4a68-ad89-39ec50792a6a: nlp_agent has been registered.
INFO:llama_agents.message_queues.simple - Consumer 94309b7e-6ffb-457f-ab0e-0c277f01bc14: human has been registered.
INFO:llama_agents.message_queues.simple - Consumer ControlPlaneServer-69c34eb2-5581-4df3-a286-e52008a35013: control_plane has been registered.
INFO:llama_agents.services.agent - calculator_agent launch_local
INFO:llama_agents.services.agent - nlp_agent launch_local
INFO:llama_agents.message_queues.base - Publishing message to 'control_plane' with action 'new_task'
INFO:llama_agents.message_queues.simple - Launching message queue locally
INFO:llama_agents.message_queues.base - Publishing message to 'calculator_agent' with action 'new_task'
INFO:llama_agents.message_queues.simple - Successfully published m

Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: calculator
Action Input: {'operation': 'divide', 'a': 100, 'b': 20}
Observation: The result of 100 divide 20 is 5.0


INFO:llama_agents.message_queues.base - Publishing message to 'control_plane' with action 'completed_task'


Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: The result of dividing 100 by 20 is 5.0.


INFO:llama_agents.message_queues.base - Publishing message to 'calculator_agent' with action 'new_task'
INFO:llama_agents.message_queues.simple - Successfully published message 'control_plane' to consumer.
INFO:llama_agents.services.agent - Created new task: d264da8e-b172-4dde-bc5f-e6400340c4e4
INFO:llama_agents.message_queues.simple - Successfully published message 'calculator_agent' to consumer.


Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: calculator
Action Input: {'operation': 'divide', 'a': 100, 'b': 20}
Observation: The result of 100 divide 20 is 5.0


INFO:llama_agents.message_queues.base - Publishing message to 'control_plane' with action 'completed_task'


Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: The result of dividing 100 by 20 is 5.0.


INFO:llama_agents.message_queues.base - Publishing message to 'human' with action 'completed_task'
INFO:llama_agents.message_queues.simple - Successfully published message 'control_plane' to consumer.
INFO:llama_agents.message_queues.simple - Successfully published message 'human' to consumer.


Result: Dividing 100 by 20 results in 5.0.


In [ ]:
# from llama_agents import ServerLauncher, CallableMessageConsumer


# # Additional human consumer
# def handle_result(message) -> None:
#     print(f"Got result:", message.data)


# human_consumer = CallableMessageConsumer(
#     handler=handle_result, message_type="human"
# )

# # Define Launcher
# launcher = ServerLauncher(
#     [agent_server_1, agent_server_2],
#     control_plane,
#     message_queue,
#     additional_consumers=[human_consumer],
# )

# # Launch it!
# launcher.launch_servers()

In [ ]:
# from llama_agents import LlamaAgentsClient, AsyncLlamaAgentsClient

# client = LlamaAgentsClient("http://127.0.0.1:8001")  # i.e. http://127.0.0.1:8001
# task_id = client.create_task("can you divide 100 by 20?")
# # <Wait a few seconds>
# # returns TaskResult or None if not finished
# result = client.get_task_result(task_id)

